# Embeddings + LSTM

In [38]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.models import Sequential
from keras.callbacks import EarlyStopping

In [2]:
data = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')
tags = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
X = data.comment_text
Y = data[tags]
data[0:10]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0
5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0
8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0
9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0


In [34]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(X))
train_tokenized = pad_sequences(tokenizer.texts_to_sequences(X), 100)
test_tokenized = pad_sequences(tokenizer.texts_to_sequences(test.comment_text), 100)

In [35]:
print('Train shape:',train_tokenized.shape)
print('Test shape:',test_tokenized.shape)

Train shape: (159571, 100)
Test shape: (153164, 100)


In [36]:
model = Sequential([
    Embedding(20000, 100, input_length=100),
    Bidirectional(LSTM(100, return_sequences=True)),
    GlobalMaxPooling1D(),
    Dense(100, activation="relu"),
    Dropout(0.1),
    Dense(6, activation="sigmoid")
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics= ["accuracy"])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 100, 100)          2000000   
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 100, 200)          160800    
_________________________________________________________________
global_max_pooling1d_17 (Glo (None, 200)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 100)               20100     
_________________________________________________________________
dropout_19 (Dropout)         (None, 100)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 6)                 606       
Total params: 2,181,506
Trainable params: 2,181,506
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(train_tokenized, Y, epochs=2)

Epoch 1/2
159571/159571 [==============================] - 1305s 8ms/step - loss: 0.0452 - acc: 0.9832
Epoch 2/2
159571/159571 [==============================] - 1251s 8ms/step - loss: 0.0395 - acc: 0.9847


In [41]:
predicted  = model.predict(test_tokenized, verbose=1)
submission = pd.DataFrame(data=predicted,columns=tags,index=test["id"] )
submission[:5]

153164/153164 [==============================] - 285s 2ms/step


,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,
00001cee341fdb12,0.998396,3.898600e-01,0.967518,7.044257e-02,0.926057,3.367495e-01
0000247867823ef7,0.000069,5.579686e-09,0.000014,8.107273e-08,0.000005,2.105901e-07
00013b17ad220c46,0.004311,6.004250e-06,0.000933,5.286001e-05,0.000648,5.334762e-05
00017563c3f7919a,0.000293,5.630267e-08,0.000062,9.748851e-07,0.000031,1.481763e-06
00017695ad8997eb,0.000238,6.071940e-08,0.000050,8.871141e-07,0.000024,1.439776e-06


In [42]:
submission.to_csv('../Submissions/embeddings-lstm.csv')

# Резултат: 0.9752